<a href="https://colab.research.google.com/github/ananyaluthra/Web-Scraping/blob/main/WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests

In [ ]:
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=50c49eb69cb0e084b03668141d96b989fcddd47bee28c7e3ced808966130520c
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
import requests
import bs4
from pathlib import Path
import json
import argparse

In [ ]:
def get_data():
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
  }
  file=Path('/content/new.html')
  if not file.exists():
    data = requests.get("https://www.montrealgazette.com", headers=headers)
    with open('new.html','w') as f:
      f.write(data.text)
  else:
    with open('new.html','r') as f:
      return f.read()

In [ ]:
def open_links(j,link):
  names=['First','Second','Third','Fourth','Fifth']
  new_file=names[j]+''+'.html'
  path='/content/'+new_file
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
  }
  file=Path(path)
  if not file.exists():
    data = requests.get(link, headers=headers)
    with open(new_file,'w') as f:
      f.write(data.text)
  else:
    with open(new_file,'r') as f:
      return f.read()


In [ ]:
def output(Output,Title,Subtitle,Author,PublishDate,para):
  final={'Title':Title,'Publication Date':PublishDate,'Author':Author,'Blurb':Subtitle}
  Output.append(final)
  Output.append(para)
  return Output



In [93]:
def main():
  parser=argparse.ArgumentParser()
  parser.add_argument('-o','--output',required=True,help="Output File")
  args=parser.parse_args()
  links=[]
  para=[]
  html_data=get_data()
  soup=bs4.BeautifulSoup(html_data,"html.parser")
  articles=soup.find_all('article',class_="article-card article-card--image-left article-card--hide-padlock")
  if articles:
    for i in range(len(articles)):
      data=json.loads(articles[i]['data-evt-val'])
      links.append(data['target_url'])
  j=0
  Final_list=[]
  for i in links:
    webpage_data=open_links(j,i)
    j+=1
    soup2=bs4.BeautifulSoup(webpage_data,"html.parser")
    divs=soup2.find('div',class_='article-header__detail__texts')
    if divs:
      Title=divs.find('h1',class_='article-title')
      Subtitle=divs.find('p',class_='article-subtitle')
      Author=divs.find('a')
      Date=divs.find('div',class_='published-date')
      if Date:
        PublishDate=Date.find('span',class_='published-date__since')
    text=soup2.find_all('section',class_="article-content__content-group article-content__content-group--story")
    if text:
      for i in text:
        article_info=i.find_all('p')
    if(article_info):
      para.append(article_info)
    result=output(Final_list,Title.text,Subtitle.text,Author.text,PublishDate.text,para)
    with open(args.output,'w') as filewrite:
      json.dump(result,filewrite)
